# Joblib

##### Joblib est une bibliothèque Python qui permet une parallélisation facile des tâches liées au processeur. Il est particulièrement utile pour les tâches qui impliquent des calculs lourds, comme le traitement de données ou l'apprentissage automatique. 

In [2]:
!pip3 install joblib



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip


La vision est de fournir des outils pour obtenir facilement de meilleures performances et une meilleure reproductibilité lorsque vous travaillez avec des travaux de longue durée.

#### Évitez de calculer deux fois la même chose :
le code est souvent réexécuté encore et encore, par exemple lors du prototypage de travaux lourds en calcul (comme dans le développement scientifique), mais les solutions artisanales pour résoudre ce problème sont sujettes aux erreurs et conduisent souvent à des résultats non reproductibles.

#### Persistance sur le disque de manière transparente : 
il est difficile de conserver efficacement des objets arbitraires contenant des données volumineuses. L'utilisation du mécanisme de mise en cache de joblib évite la persistance manuscrite et lie implicitement le fichier sur disque au contexte d'exécution de l'objet Python d'origine. En conséquence, la persistance de joblib est bonne pour reprendre un statut d'application ou un travail de calcul, par exemple après un crash.

# 1) Assistant parallèle embarrassant

## Traitement légers 

In [5]:
import time
from joblib import Parallel, delayed

# Fonction pour effectuer une opération simple (multiplication ici)
def operation_simple(item):
    for i in 
    return item * 2

# Données à traiter
data = list(range(1000000))

# Sans parallélisation (exécution séquentielle)
start_time = time.time()

resultats_sans_parallel = [operation_simple(item) for item in data]

end_time = time.time()
temps_execution_sans_parallel = end_time - start_time

# En utilisant Joblib pour paralléliser l'opération
start_time = time.time()

resultats_avec_parallel = Parallel(n_jobs=12)(delayed(operation_simple)(item) for item in data)

end_time = time.time()
temps_execution_avec_parallel = end_time - start_time

# Comparaison des temps d'exécution
print("Temps d'exécution sans parallélisation:", temps_execution_sans_parallel)
print("Temps d'exécution avec parallélisation:", temps_execution_avec_parallel)

Temps d'exécution sans parallélisation: 0.10040926933288574
Temps d'exécution avec parallélisation: 4.4656970500946045


!!! Le temps d'exécution sans parallélisation étant nettement inférieur au temps d'exécution avec parallélisation dans notre exemple, parceque la tâche simple utilisée dans le code est trop petite pour montrer l'avantage de la parallélisation. En effet, la création des tâches parallèles et leur distribution entre les cœurs peuvent engendrer un certain surcoût qui devient significatif lorsque la tâche est petite

## Traitement Lourd

In [10]:
import time
from joblib import Parallel, delayed

# Fonction pour effectuer une tâche intensive en calculs (factorielle ici)
def factorielle(n):
    result = 1
    for i in range(1, n+1):
        result *= 99999999999999999999999999999999999*i
    return result

# Données à traiter
data = list(range(1, 1001))  # Calculer factorielle de 1 à 1000

# Sans parallélisation (exécution séquentielle)
start_time = time.time()

resultats_sans_parallel = [factorielle(n) for n in data]

end_time = time.time()
temps_execution_sans_parallel = end_time - start_time

# En utilisant Joblib pour paralléliser l'opération
start_time = time.time()

resultats_avec_parallel = Parallel(n_jobs=-1)(delayed(factorielle)(n) for n in data)

end_time = time.time()
temps_execution_avec_parallel = end_time - start_time

# Comparaison des temps d'exécution
print("Temps d'exécution sans parallélisation:", temps_execution_sans_parallel)
print("\nTemps d'exécution avec parallélisation:", temps_execution_avec_parallel)


Temps d'exécution sans parallélisation: 2.035102367401123

Temps d'exécution avec parallélisation: 0.4305577278137207


# 2) Mise en cache disque transparente et rapide de la valeur de sortie

Joblib fournit un mécanisme de mise en cache pour mettre en cache de manière transparente les résultats des appels de fonctions coûteux sur le disque, permettant une mémorisation efficace (réévaluation paresseuse) de ces fonctions. Cela peut considérablement accélérer le calcul en évitant d'avoir à recalculer les résultats pour les mêmes arguments d'entrée. Pour utiliser Joblib pour la mise en cache disque transparente des fonctions et la mémorisation, nous suivrons ces étapes :


In [25]:
import time
from joblib import Memory

# Create a Joblib Memory object with caching enabled
cache_dir = "/cache_dir"
memory = Memory(location=cache_dir, verbose=0)

# Define the Fibonacci function with caching enabled
@memory.cache
def fibonacci_with_cache(n):
    if n <= 1:
        return n
    return fibonacci_with_cache(n-1) + fibonacci_with_cache(n-2)

# Define the Fibonacci function without caching
def fibonacci_without_cache(n):
    if n <= 1:
        return n
    return fibonacci_without_cache(n-1) + fibonacci_without_cache(n-2)

# Compare the time execution with and without caching
n = 30

# Time execution with caching enabled
start_time = time.time()
result_with_cache = fibonacci_with_cache(n)
end_time = time.time()
time_with_cache = end_time - start_time

# Time execution without caching
start_time = time.time()
result_without_cache = fibonacci_without_cache(n)
end_time = time.time()
time_without_cache = end_time - start_time

# Print the results
print("Fibonacci with caching: Result =", result_with_cache, "Time =", time_with_cache, "seconds")
print("\nFibonacci without caching: Result =", result_without_cache, "Time =", time_without_cache, "seconds")

Fibonacci with caching: Result = 832040 Time = 0.0022644996643066406 seconds

Fibonacci without caching: Result = 832040 Time = 0.19536638259887695 seconds


# Dask

## dask.bag

In [30]:
# !pip install dask

### Without dask.bag

In [27]:
import dask
from dask import delayed, compute

In [28]:
import time

def sequential_sum(numbers):
    total_sum = 0
    for num in numbers:
        total_sum += num
    return total_sum

# Create a large list of numbers
large_list = list(range(1, 10000001))

start_time = time.time()
result = sequential_sum(large_list)
end_time = time.time()

print(f"Total sum: {result}")
print(f"Time taken (sequential): {end_time - start_time} seconds")

Total sum: 50000005000000
Time taken (sequential): 0.30614256858825684 seconds


### With dask.bag

In [29]:
import time
import dask
import dask.bag as db

def parallel_sum(numbers):
    return sum(numbers)

# Create a Dask bag from the large list of numbers
large_bag = db.from_sequence(list(range(1, 10000001)))

start_time = time.time()
result = dask.compute(parallel_sum(large_bag))[0]
end_time = time.time()

print(f"Total sum: {result}")
print(f"Time taken (parallel with Dask): {end_time - start_time} seconds")


Total sum: 50000005000000
Time taken (parallel with Dask): 8.647001028060913 seconds


 ## Dask delayed function

### Without delayed function

In [47]:
import time

def sequential_mul(numbers):
    total_mul = 1
    for num in numbers:
        total_mul *= num
    return total_mul

# Create a large list of numbers
large_list = list(range(1, 10000))

start_time = time.time()
result = sequential_sum(large_list)
end_time = time.time()

# print(f"Total mul: {result}")
print(f"Time taken (sequential): {end_time - start_time} seconds")

Time taken (sequential): 0.04382729530334473 seconds


### With delayed function

In [48]:
import time
import dask
from dask import delayed

def sequential_mul(numbers):
    total_mul = 1
    for num in numbers:
        total_mul *= num
    return total_mul

@delayed
def delayed_mul(numbers):
    return sequential_mul(numbers)

# Create a large list of numbers
large_list = list(range(1, 10000))

# Create a list of delayed tasks
delayed_tasks = [delayed_mul(large_list)]

start_time = time.time()

# Trigger the computation by calling dask.compute on the list of delayed tasks
result = dask.compute(*delayed_tasks)

end_time = time.time()

# print(f"Total sum: {result[0]}")
print(f"Time taken (parallel with Dask): {end_time - start_time} seconds")


Time taken (parallel with Dask): 0.03241610527038574 seconds
